In [175]:

import numpy as np
import pandas as pd
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


# Content Based Fashion Recommender

Data Collection and pre processing

In [176]:
fashion_data =pd.read_csv('myntra_products_catalog.csv')

In [177]:
fashion_data['index'] = fashion_data.index


In [178]:
fashion_data

,ProductID,ProductName,ProductBrand,Gender,Price (INR),NumImages,Description,PrimaryColor,index
0,10017413,DKNY Unisex Black & Grey Printed Medium Trolle...,DKNY,Unisex,11745,7,"Black and grey printed medium trolley bag, sec...",Black,0
1,10016283,EthnoVogue Women Beige & Grey Made to Measure ...,EthnoVogue,Women,5810,7,Beige & Grey made to measure kurta with churid...,Beige,1
2,10009781,SPYKAR Women Pink Alexa Super Skinny Fit High-...,SPYKAR,Women,899,7,Pink coloured wash 5-pocket high-rise cropped ...,Pink,2
3,10015921,Raymond Men Blue Self-Design Single-Breasted B...,Raymond,Men,5599,5,Blue self-design bandhgala suitBlue self-desig...,Blue,3
4,10017833,Parx Men Brown & Off-White Slim Fit Printed Ca...,Parx,Men,759,5,"Brown and off-white printed casual shirt, has ...",White,4
...,...,...,...,...,...,...,...,...,...
12486,10262843,Pepe Jeans Men Black Hammock Slim Fit Low-Rise...,Pepe Jeans,Men,1299,7,"Black dark wash 5-pocket low-rise jeans, clean...",Black,12486
12487,10261721,Mochi Women Gold-Toned Solid Heels,Mochi,Women,1990,5,"A pair of gold-toned open toe heels, has regul...",Gold,12487
12488,10261607,612 league Girls Navy Blue & White Printed Reg...,612 league,Girls,602,4,Navy Blue and White printed mid-rise denim sho...,Blue,12488
12489,10266621,Bvlgari Men Aqva Pour Homme Marine Eau de Toil...,Bvlgari,Men,8950,2,Bvlgari Men Aqva Pour Homme Marine Eau de Toil...,NaN,12489


In [179]:
fashion_data.head()

,ProductID,ProductName,ProductBrand,Gender,Price (INR),NumImages,Description,PrimaryColor,index
0,10017413,DKNY Unisex Black & Grey Printed Medium Trolle...,DKNY,Unisex,11745,7,"Black and grey printed medium trolley bag, sec...",Black,0
1,10016283,EthnoVogue Women Beige & Grey Made to Measure ...,EthnoVogue,Women,5810,7,Beige & Grey made to measure kurta with churid...,Beige,1
2,10009781,SPYKAR Women Pink Alexa Super Skinny Fit High-...,SPYKAR,Women,899,7,Pink coloured wash 5-pocket high-rise cropped ...,Pink,2
3,10015921,Raymond Men Blue Self-Design Single-Breasted B...,Raymond,Men,5599,5,Blue self-design bandhgala suitBlue self-desig...,Blue,3
4,10017833,Parx Men Brown & Off-White Slim Fit Printed Ca...,Parx,Men,759,5,"Brown and off-white printed casual shirt, has ...",White,4


In [180]:
fashion_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12491 entries, 0 to 12490
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   ProductID     12491 non-null  int64 
 1   ProductName   12491 non-null  object
 2   ProductBrand  12491 non-null  object
 3   Gender        12491 non-null  object
 4   Price (INR)   12491 non-null  int64 
 5   NumImages     12491 non-null  int64 
 6   Description   12491 non-null  object
 7   PrimaryColor  11597 non-null  object
 8   index         12491 non-null  int64 
dtypes: int64(4), object(5)
memory usage: 878.4+ KB


In [181]:
fashion_data.shape

(12491, 9)

feature Selection

In [182]:
selected_features = ['ProductName','ProductBrand','Gender','Description','PrimaryColor']
selected_features

['ProductName', 'ProductBrand', 'Gender', 'Description', 'PrimaryColor']

In [183]:
fashion_data[fashion_data['ProductName']==' ']
# No null values in dataset

,ProductID,ProductName,ProductBrand,Gender,Price (INR),NumImages,Description,PrimaryColor,index


In [184]:
distinct_count = fashion_data['ProductBrand'].value_counts()
distinct_count


Indian Terrain                  971
Puma                            345
Pepe Jeans                      340
AURELIA                         307
Flying Machine                  301
                               ... 
Johnny Bravo by Kook N Keech      1
Allen Solly Sport                 1
HAMMER                            1
CUTECUMBER                        1
Stupid Cupid                      1
Name: ProductBrand, Length: 677, dtype: int64

In [185]:
distinct_count = fashion_data['ProductName'].value_counts()
distinct_count

Parx Men Blue Slim Fit Checked Casual Shirt                                          16
ZHEIA Women Blue Skinny Fit Mid-Rise Clean Look Stretchable Jeans                    15
Roadster Men Blue Skinny Fit Mid-Rise Clean Look Stretchable Jeans                   14
Sonari Pack of 2 Full-Coverage Bras                                                  14
Roadster Men Blue Slim Fit Mid-Rise Clean Look Stretchable Jeans                     13
                                                                                     ..
HRX by Hrithik Roshan Ultralyte Men Navy Blue Solid Running T-shirt                   1
Flying Machine Men Blue Michael Tapered Fit Mid-Rise Clean Look Stretchable Jeans     1
FAUSTO Men Red Slip-On Sneakers                                                       1
Indian Terrain Boys Rust Brown Printed Round Neck T-shirt                             1
Pepe Jeans Men Black & Grey Striped Polo Collar T-shirt                               1
Name: ProductName, Length: 10761

In [186]:
distinct_count = fashion_data['Gender'].value_counts()
distinct_count

Women          5126
Men            4591
Unisex         1188
Boys           1100
Girls           440
Unisex Kids      46
Name: Gender, dtype: int64

In [187]:
ind1 = list(fashion_data['Gender'].loc[lambda x: x=='Unisex Kids'].index)
ind1

[2078,
 2083,
 2132,
 2135,
 2144,
 2198,
 2230,
 2264,
 2332,
 2472,
 2478,
 2548,
 2578,
 2580,
 2591,
 2610,
 2627,
 2631,
 2664,
 2686,
 2755,
 2758,
 2816,
 2826,
 2848,
 2851,
 2892,
 2919,
 2928,
 2959,
 2991,
 3351,
 3668,
 3797,
 3843,
 3865,
 4941,
 5341,
 5692,
 6347,
 7227,
 7329,
 7395,
 7685,
 7819,
 9803]

In [188]:
fashion_data.drop(ind1, axis=0, inplace=True)


In [189]:
distinct_count = fashion_data['Gender'].value_counts()
distinct_count

Women     5126
Men       4591
Unisex    1188
Boys      1100
Girls      440
Name: Gender, dtype: int64

In [190]:
distinct_count = fashion_data['PrimaryColor'].value_counts()
distinct_count

Blue         3429
 Black       1635
 Red         1537
 Green        903
 White        877
 Grey         677
 Brown        472
 Yellow       404
 Pink         391
 Gold         236
 Beige        235
 Maroon       186
 Orange       129
 Silver       111
 Purple        65
 Burgundy      64
 Khaki         56
 Navy          54
 Lavender      19
 Matte         17
 Mustard       15
 Magenta       13
 Charcoal      10
 Rose           8
 Platinum       3
 Bronze         3
 Peach          2
Name: PrimaryColor, dtype: int64

In [191]:
len(fashion_data)

12445

In [192]:
fashion_data.head(1)

,ProductID,ProductName,ProductBrand,Gender,Price (INR),NumImages,Description,PrimaryColor,index
0,10017413,DKNY Unisex Black & Grey Printed Medium Trolle...,DKNY,Unisex,11745,7,"Black and grey printed medium trolley bag, sec...",Black,0


In [193]:
fashion_data['ProductName'][0]

'DKNY Unisex Black & Grey Printed Medium Trolley Bag'

In [194]:
fashion_data['ProductBrand'][0]

'DKNY'

In [195]:
fashion_data['PrimaryColor'][0]

' Black'

In [196]:
fashion_data['Gender'][0]

'Unisex'

In [197]:
fashion_data['Description'][0]

'Black and grey printed medium trolley bag, secured with a TSA lockOne handle on the top and one on the side, has a trolley with a retractable handle on the top and four corner mounted inline skate wheelsOne main zip compartment, zip lining, two compression straps with click clasps, one zip compartment on the flap with three zip pocketsWarranty: 5 yearsWarranty provided by Brand Owner / Manufacturer'

In [198]:
fashion_data['PrimaryColor'].shape

(12445,)

In [199]:
for i in selected_features:
    print(i,fashion_data[i].isna().sum())

ProductName 0
ProductBrand 0
Gender 0
Description 0
PrimaryColor 894


In [200]:
ind1 = list(fashion_data['PrimaryColor'].loc[lambda x: x.isna()].index)
fashion_data.drop(ind1, axis=0, inplace=True)

In [201]:
fashion_data['PrimaryColor'].shape

(11551,)

In [202]:
fashion_data['PrimaryColor'].shape

(11551,)

In [203]:
# ind1 = list(fashion_data['ProductName'].loc[lambda x: " "].index)
# print(ind1)
# fashion_data.drop(ind1, axis=0, inplace=True)

KeyError: ' '

In [205]:
fashion_data.shape

(11551, 9)

In [206]:
fashion_data['ProductName'].replace(' ', np.nan, inplace=True)
fashion_data.dropna(subset=['ProductName'], inplace=True)

In [207]:
fashion_data.shape

(11551, 9)

In [208]:
fashion_data['PrimaryColor'].shape

(11551,)

product name is already a combination of productBrand + gender + color + "name"

In [209]:
fashion_data['ProductName']=fashion_data['ProductName'].apply(str.lower)
fashion_data['ProductBrand']=fashion_data['ProductBrand'].apply(str.lower)
fashion_data['PrimaryColor']=fashion_data['PrimaryColor'].apply(str.lower)
fashion_data['Gender']=fashion_data['Gender'].apply(str.lower)
fashion_data['Description']=fashion_data['Description'].apply(str.lower)

In [210]:
cnt =0
ind1 = []
ind2=[]
for i in range(len(fashion_data)):
    try:
        productNAME = fashion_data['ProductName'][i].split(' ')
        productBRAND = fashion_data['ProductBrand'][i].split(' ')
#         print(productNAME,productBRAND)
        if(productNAME[0]!=productBRAND[0]):
            print(productNAME[0],productBRAND[0])
            print(i)
            ind2.append(i)
     
    except KeyError:
        cnt+=1
        ind1.append(i)

    

goldea bvlgari
307
brown u.s.
438
blue gas
814
black beli
864
black mango
964
wishful w
2477
wishful w
2583
women studio
7176
22 accessher
7991
area rugsmith
9150
u.s.polo u.s.
9692
area rugsmith
9867
van v
9994
van v
10293
van v
10595
odcroft oddcroft
10866
pepe oshkosh
11205


In [211]:
# there are no rows whose product brand is missed in product name
ind2

[307,
 438,
 814,
 864,
 964,
 2477,
 2583,
 7176,
 7991,
 9150,
 9692,
 9867,
 9994,
 10293,
 10595,
 10866,
 11205]

In [212]:
fashion_data.iloc[ind2]


,ProductID,ProductName,ProductBrand,Gender,Price (INR),NumImages,Description,PrimaryColor,index
327,10017917,parx men red & black slim fit printed casual s...,parx,men,639,5,"red and black printed casual shirt, has a spre...",red,327
469,10000675,lady lyka women black and fuschia pack of 2 be...,lady lyka,women,425,5,pack of 2black and fuschia solid medium-covera...,black,469
866,10017663,parx men pink slim fit printed casual shirt,parx,men,679,5,"pink printed casual shirt, has a spread collar...",pink,866
919,10003797,homesake gold-toned solid handcrafted table la...,homesake,unisex,1732,5,type: handcrafted table lamp with shadecolour:...,gold,919
1025,10019811,carlton london women coral orange solid cinche...,carlton london,women,625,5,coral orange solid woven cinched waist top wit...,orange,1025
2709,10051243,vishudh peach-coloured & grey striped tunic,vishudh,women,601,5,"peach-coloured and grey printed tunic, has a s...",red,2709
2822,10039475,la loft women off-white floral printed shirt s...,la loft,women,611,5,off-white and blue printed woven shirt style ...,white,2822
7707,10188279,wrogn men navy blue solid slim fit round neck ...,wrogn,men,1299,5,"navy blue solid t-shirt, has a round neck, lon...",blue,7707
8577,10202731,van heusen woman women white regular fit solid...,van heusen woman,women,1499,5,"white solid casual shirt, has a spread collar,...",white,8577
9810,10232563,rugsmith multicoloured premium modern pattern rug,rugsmith,unisex,1799,3,content: one runner colour: navy blue and mult...,red,9810


In [213]:
fashion_data.drop(ind2, axis=0, inplace=True)


In [214]:
cnt

830

In [215]:
cnt =0
ind1 = []
for i in range(len(fashion_data)):
    try:
        
        productNAME = fashion_data['ProductName'][i].split(' ')
        productBRAND = fashion_data['ProductBrand'][i].split(' ')

        if(productNAME[len(productBRAND)]!=fashion_data['Gender'][0]):
#             print(i)
            print(productNAME,productBRAND,fashion_data['Gender'][0]) 
            productNAME[len(productBRAND)] = ""
#             productNAME.insert(len(productBRAND), fashion_data['Gender'])
#             print(productNAME,productBRAND)
            print(productNAME)
            t = " ".join(productNAME).split()
            t =  " ".join(productNAME)
            print(t)
            fashion_data['ProductName'][i] = t
     
    except KeyError:
        cnt+=1
        ind1.append(i)


['ethnovogue', 'women', 'beige', '&', 'grey', 'made', 'to', 'measure', 'custom', 'made', 'kurta', 'set', 'with', 'jacket'] ['ethnovogue'] unisex
['ethnovogue', '', 'beige', '&', 'grey', 'made', 'to', 'measure', 'custom', 'made', 'kurta', 'set', 'with', 'jacket']
ethnovogue  beige & grey made to measure custom made kurta set with jacket
['spykar', 'women', 'pink', 'alexa', 'super', 'skinny', 'fit', 'high-rise', 'clean', 'look', 'stretchable', 'cropped', 'jeans'] ['spykar'] unisex
['spykar', '', 'pink', 'alexa', 'super', 'skinny', 'fit', 'high-rise', 'clean', 'look', 'stretchable', 'cropped', 'jeans']
spykar  pink alexa super skinny fit high-rise clean look stretchable cropped jeans
['raymond', 'men', 'blue', 'self-design', 'single-breasted', 'bandhgala', 'suit'] ['raymond'] unisex
['raymond', '', 'blue', 'self-design', 'single-breasted', 'bandhgala', 'suit']
raymond  blue self-design single-breasted bandhgala suit
['parx', 'men', 'brown', '&', 'off-white', 'slim', 'fit', 'printed', 'cas

C:\Users\ajitg\AppData\Local\Temp\ipykernel_27856\2204936945.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fashion_data['ProductName'][i] = t


['sej', 'by', 'nisha', 'gupta', 'set', 'of', '6', 'yellow', '&', 'multicoloured', 'printed', 'table', 'placemats'] ['sej', 'by', 'nisha', 'gupta'] unisex
['sej', 'by', 'nisha', 'gupta', '', 'of', '6', 'yellow', '&', 'multicoloured', 'printed', 'table', 'placemats']
sej by nisha gupta  of 6 yellow & multicoloured printed table placemats
['accessher', 'gold-plated', '&', 'red', 'meenakari', 'maang', 'tika'] ['accessher'] unisex
['accessher', '', '&', 'red', 'meenakari', 'maang', 'tika']
accessher  & red meenakari maang tika
['sera', 'women', 'yellow', '&', 'off-white', 'printed', 'kurta', 'with', 'palazzos'] ['sera'] unisex
['sera', '', 'yellow', '&', 'off-white', 'printed', 'kurta', 'with', 'palazzos']
sera  yellow & off-white printed kurta with palazzos
['tayhaa', 'black', '&', 'white', 'square', 'photo', 'frame'] ['tayhaa'] unisex
['tayhaa', '', '&', 'white', 'square', 'photo', 'frame']
tayhaa  & white square photo frame
['park', 'avenue', 'women', 'blue', 'solid', 'a-line', 'top'] ['

['sej', 'by', 'nisha', 'gupta', 'set', 'of', '6', 'multicoloured', 'printed', 'table', 'placemats'] ['sej', 'by', 'nisha', 'gupta'] unisex
['sej', 'by', 'nisha', 'gupta', '', 'of', '6', 'multicoloured', 'printed', 'table', 'placemats']
sej by nisha gupta  of 6 multicoloured printed table placemats
['parfait', 'plus', 'size', 'burgundy', 'lace', 'underwired', 'lightly', 'padded', 'plunge', 'bra', 'p5351'] ['parfait'] unisex
['parfait', '', 'size', 'burgundy', 'lace', 'underwired', 'lightly', 'padded', 'plunge', 'bra', 'p5351']
parfait  size burgundy lace underwired lightly padded plunge bra p5351
['gas', 'men', 'maroon', '&', 'navy', 'blue', 'slim', 'fit', 'printed', 'casual', 'shirt'] ['gas'] unisex
['gas', '', 'maroon', '&', 'navy', 'blue', 'slim', 'fit', 'printed', 'casual', 'shirt']
gas  maroon & navy blue slim fit printed casual shirt
['homesake', 'gold-toned', 'solid', 'handcrafted', 'table', 'lamp', 'with', 'shade'] ['homesake'] unisex
['homesake', '', 'solid', 'handcrafted', 'ta

['geox', 'men', 'grey', 'solid', 'suede', 'sneakers'] ['geox'] unisex
['geox', '', 'grey', 'solid', 'suede', 'sneakers']
geox  grey solid suede sneakers
['here&now', 'women', 'black', 'solid', 'biker', 'jacket'] ['here&now'] unisex
['here&now', '', 'black', 'solid', 'biker', 'jacket']
here&now  black solid biker jacket
['soumya', 'multicoloured', 'set', 'of', '5', 'floral', 'square', 'cushion', 'covers'] ['soumya'] unisex
['soumya', '', 'set', 'of', '5', 'floral', 'square', 'cushion', 'covers']
soumya  set of 5 floral square cushion covers
['geox', 'men', 'black', 'leather', 'driving', 'shoes'] ['geox'] unisex
['geox', '', 'black', 'leather', 'driving', 'shoes']
geox  black leather driving shoes
['ether', 'men', 'navy', 'blue', 'colorblocked', 'oxford', 'polo', 't-shirt'] ['ether'] unisex
['ether', '', 'navy', 'blue', 'colorblocked', 'oxford', 'polo', 't-shirt']
ether  navy blue colorblocked oxford polo t-shirt
['sej', 'by', 'nisha', 'gupta', 'yellow', '&', 'grey', 'printed', 'tissue',

...

['van', 'heusen', 'woman', 'women', 'black', 'woven', 'design', 'pumps'] ['van', 'heusen', 'woman'] unisex
['van', 'heusen', 'woman', '', 'black', 'woven', 'design', 'pumps']
van heusen woman  black woven design pumps
['w', 'women', 'blue', 'embroidered', 'kurta', 'with', 'patiala', 'salwar'] ['w'] unisex
['w', '', 'blue', 'embroidered', 'kurta', 'with', 'patiala', 'salwar']
w  blue embroidered kurta with patiala salwar
['moda', 'rapido', 'women', 'navy', 'blue', '&', 'white', 'printed', 'a-line', 'kurta'] ['moda', 'rapido'] unisex
['moda', 'rapido', '', 'navy', 'blue', '&', 'white', 'printed', 'a-line', 'kurta']
moda rapido  navy blue & white printed a-line kurta
['daily', 'life', 'forever52', 'women', 'purple', 'super', 'stay', 'lip', 'gloss', '7ml'] ['daily', 'life', 'forever52'] unisex
['daily', 'life', 'forever52', '', 'purple', 'super', 'stay', 'lip', 'gloss', '7ml']
daily life forever52  purple super stay lip gloss 7ml
['fabnest', 'women', 'mustard', 'yellow', 'solid', 'kurti'] 

['inc', '5', 'women', 'gold-toned', 'solid', 'one', 'toe', 'flats'] ['inc', '5'] unisex
['inc', '5', '', 'gold-toned', 'solid', 'one', 'toe', 'flats']
inc 5  gold-toned solid one toe flats
['here&now', 'men', 'white', '&', 'blue', 'slim', 'fit', 'checked', 'casual', 'shirt'] ['here&now'] unisex
['here&now', '', 'white', '&', 'blue', 'slim', 'fit', 'checked', 'casual', 'shirt']
here&now  white & blue slim fit checked casual shirt
['aj', 'dezines', 'boys', 'cream-coloured', 'solid', 'kurta', 'with', 'pyjamas'] ['aj', 'dezines'] unisex
['aj', 'dezines', '', 'cream-coloured', 'solid', 'kurta', 'with', 'pyjamas']
aj dezines  cream-coloured solid kurta with pyjamas
['proline', 'active', 'men', 'grey', 'solid', 'slim', 'fit', 'track', 'pants'] ['proline', 'active'] unisex
['proline', 'active', '', 'grey', 'solid', 'slim', 'fit', 'track', 'pants']
proline active  grey solid slim fit track pants
['basics', 'men', 'teal', 'blue', 'slim', 'fit', 'checked', 'casual', 'shirt'] ['basics'] unisex
['b

['ecko', 'unltd', 'men', 'navy', 'blue', 'striped', 'polo', 'collar', 't-shirt'] ['ecko', 'unltd'] unisex
['ecko', 'unltd', '', 'navy', 'blue', 'striped', 'polo', 'collar', 't-shirt']
ecko unltd  navy blue striped polo collar t-shirt
['raymond', 'men', 'mustard', 'yellow', 'regular', 'fit', 'solid', 'formal', 'shirt'] ['raymond'] unisex
['raymond', '', 'mustard', 'yellow', 'regular', 'fit', 'solid', 'formal', 'shirt']
raymond  mustard yellow regular fit solid formal shirt
['louis', 'philippe', 'men', 'black', 'slim', 'fit', 'solid', 'regular', 'trousers'] ['louis', 'philippe'] unisex
['louis', 'philippe', '', 'black', 'slim', 'fit', 'solid', 'regular', 'trousers']
louis philippe  black slim fit solid regular trousers
['flying', 'machine', 'boys', 'yellow', 'and', 'colourblocked', 'polo', 'collar', 't-shirt'] ['flying', 'machine'] unisex
['flying', 'machine', '', 'yellow', 'and', 'colourblocked', 'polo', 'collar', 't-shirt']
flying machine  yellow and colourblocked polo collar t-shirt
[

['sg', 'yuvraj', 'boys', 'beige', 'solid', 'kurta', 'with', 'pyjamas'] ['sg', 'yuvraj'] unisex
['sg', 'yuvraj', '', 'beige', 'solid', 'kurta', 'with', 'pyjamas']
sg yuvraj  beige solid kurta with pyjamas
['alcis', 'women', 'grey', 'solid', 'sporty', 'jacket'] ['alcis'] unisex
['alcis', '', 'grey', 'solid', 'sporty', 'jacket']
alcis  grey solid sporty jacket
['pepe', 'jeans', 'boys', 'blue', 'regular', 'fit', 'solid', 'casual', 'shirt'] ['pepe', 'jeans'] unisex
['pepe', 'jeans', '', 'blue', 'regular', 'fit', 'solid', 'casual', 'shirt']
pepe jeans  blue regular fit solid casual shirt
['sg', 'yuvraj', 'boys', 'beige', 'solid', 'kurta', 'with', 'pyjamas'] ['sg', 'yuvraj'] unisex
['sg', 'yuvraj', '', 'beige', 'solid', 'kurta', 'with', 'pyjamas']
sg yuvraj  beige solid kurta with pyjamas
['sg', 'yuvraj', 'boys', 'coffee', 'brown', 'solid', 'kurta', 'with', 'pyjamas'] ['sg', 'yuvraj'] unisex
['sg', 'yuvraj', '', 'coffee', 'brown', 'solid', 'kurta', 'with', 'pyjamas']
sg yuvraj  coffee brown s

['lino', 'perros', 'blue', 'solid', 'sling', 'bag'] ['lino', 'perros'] unisex
['lino', 'perros', '', 'solid', 'sling', 'bag']
lino perros  solid sling bag
['lapis', 'o', 'lupo', 'blue', 'solid', 'handheld', 'bag'] ['lapis', 'o', 'lupo'] unisex
['lapis', 'o', 'lupo', '', 'solid', 'handheld', 'bag']
lapis o lupo  solid handheld bag
['pepe', 'jeans', 'boys', 'off-white', 'printed', 'hooded', 'sweatshirt'] ['pepe', 'jeans'] unisex
['pepe', 'jeans', '', 'off-white', 'printed', 'hooded', 'sweatshirt']
pepe jeans  off-white printed hooded sweatshirt
['lapis', 'o', 'lupo', 'women', 'beige', 'textured', 'handheld', 'bag'] ['lapis', 'o', 'lupo'] unisex
['lapis', 'o', 'lupo', '', 'beige', 'textured', 'handheld', 'bag']
lapis o lupo  beige textured handheld bag
['pepe', 'jeans', 'boys', 'navy', 'blue', 'checked', 'casual', 'shirt'] ['pepe', 'jeans'] unisex
['pepe', 'jeans', '', 'navy', 'blue', 'checked', 'casual', 'shirt']
pepe jeans  navy blue checked casual shirt
['genius18', 'women', 'navy', 'b

In [216]:
fashion_data.head()

,ProductID,ProductName,ProductBrand,Gender,Price (INR),NumImages,Description,PrimaryColor,index
0,10017413,dkny unisex black & grey printed medium trolle...,dkny,unisex,11745,7,"black and grey printed medium trolley bag, sec...",black,0
1,10016283,ethnovogue beige & grey made to measure custo...,ethnovogue,women,5810,7,beige & grey made to measure kurta with churid...,beige,1
2,10009781,spykar pink alexa super skinny fit high-rise ...,spykar,women,899,7,pink coloured wash 5-pocket high-rise cropped ...,pink,2
3,10015921,raymond blue self-design single-breasted band...,raymond,men,5599,5,blue self-design bandhgala suitblue self-desig...,blue,3
4,10017833,parx brown & off-white slim fit printed casua...,parx,men,759,5,"brown and off-white printed casual shirt, has ...",white,4


In [217]:
df = fashion_data[['index','ProductName','Description','Gender','PrimaryColor','ProductBrand']] 

In [219]:
df.head()

,index,ProductName,Description,Gender,PrimaryColor,ProductBrand
0,0,dkny unisex black & grey printed medium trolle...,"black and grey printed medium trolley bag, sec...",unisex,black,dkny
1,1,ethnovogue beige & grey made to measure custo...,beige & grey made to measure kurta with churid...,women,beige,ethnovogue
2,2,spykar pink alexa super skinny fit high-rise ...,pink coloured wash 5-pocket high-rise cropped ...,women,pink,spykar
3,3,raymond blue self-design single-breasted band...,blue self-design bandhgala suitblue self-desig...,men,blue,raymond
4,4,parx brown & off-white slim fit printed casua...,"brown and off-white printed casual shirt, has ...",men,white,parx


In [220]:
import csv  
df.to_csv('file2.csv')


In [221]:
df

,index,ProductName,Description,Gender,PrimaryColor,ProductBrand
0,0,dkny unisex black & grey printed medium trolle...,"black and grey printed medium trolley bag, sec...",unisex,black,dkny
1,1,ethnovogue beige & grey made to measure custo...,beige & grey made to measure kurta with churid...,women,beige,ethnovogue
2,2,spykar pink alexa super skinny fit high-rise ...,pink coloured wash 5-pocket high-rise cropped ...,women,pink,spykar
3,3,raymond blue self-design single-breasted band...,blue self-design bandhgala suitblue self-desig...,men,blue,raymond
4,4,parx brown & off-white slim fit printed casua...,"brown and off-white printed casual shirt, has ...",men,white,parx
...,...,...,...,...,...,...
12485,12485,laceandme black lace non-wired lightly padded ...,black lace full-coverage bralette bra lightly ...,women,red,laceandme
12486,12486,pepe jeans men black hammock slim fit low-rise...,"black dark wash 5-pocket low-rise jeans, clean...",men,black,pepe jeans
12487,12487,mochi women gold-toned solid heels,"a pair of gold-toned open toe heels, has regul...",women,gold,mochi
12488,12488,612 league girls navy blue & white printed reg...,navy blue and white printed mid-rise denim sho...,girls,blue,612 league


In [222]:
# combining all the selected features
# ['ProductName','ProductBrand','Gender','Description','PrimaryColor']

combined_features = fashion_data['ProductName'] + fashion_data['Description'] + fashion_data['Gender'] + fashion_data['PrimaryColor'] + fashion_data['ProductBrand'] 

In [223]:
combined_features

0        dkny unisex black & grey printed medium trolle...
1        ethnovogue  beige & grey made to measure custo...
2        spykar  pink alexa super skinny fit high-rise ...
3        raymond  blue self-design single-breasted band...
4        parx  brown & off-white slim fit printed casua...
                               ...                        
12485    laceandme black lace non-wired lightly padded ...
12486    pepe jeans men black hammock slim fit low-rise...
12487    mochi women gold-toned solid heelsa pair of go...
12488    612 league girls navy blue & white printed reg...
12490    pepe jeans men black & grey striped polo colla...
Length: 11534, dtype: object

In [224]:
# converting the text data to feature vectors

vectorizer = TfidfVectorizer()

In [225]:
feature_vectors = vectorizer.fit_transform(combined_features)

In [226]:
print(feature_vectors)

  (0, 1414)	0.15235941534048894
  (0, 6914)	0.10374749958125903
  (0, 8050)	0.10871775020224282
  (0, 1872)	0.056667515354053816
  (0, 2212)	0.052426899205398614
  (0, 9132)	0.05720602963735132
  (0, 12860)	0.09556622479238813
  (0, 8868)	0.15392528074750478
  (0, 11657)	0.06434693316800244
  (0, 4335)	0.10614595336379726
  (0, 2575)	0.1403653297718806
  (0, 2601)	0.13473744162861384
  (0, 11172)	0.09367918475632775
  (0, 3009)	0.13806115508278996
  (0, 12029)	0.08159784825099448
  (0, 6565)	0.09338164481498572
  (0, 2965)	0.1819096369526213
  (0, 13065)	0.22227522346202352
  (0, 6835)	0.08147696501964025
  (0, 12431)	0.13037211244426508
  (0, 10503)	0.13806115508278996
  (0, 5875)	0.1387994643648648
  (0, 7379)	0.12988905397475645
  (0, 3092)	0.13413545954650719
  (0, 4517)	0.10746529864423535
  :	:
  (11533, 3370)	0.21823597084380233
  (11533, 3497)	0.13981181945267915
  (11533, 10909)	0.164257915575988
  (11533, 4140)	0.15041742339992356
  (11533, 10577)	0.144636444447576
  (11533, 

COSINE SIMILARITY

In [227]:
# getting the similarity scores using cosine similarity

similarity = cosine_similarity(feature_vectors)
similarity

array([[1.        , 0.04254845, 0.03873111, ..., 0.02423677, 0.02384061,
        0.03669008],
       [0.04254845, 1.        , 0.00822072, ..., 0.00519991, 0.0074029 ,
        0.01574003],
       [0.03873111, 0.00822072, 1.        , ..., 0.00790486, 0.04362236,
        0.08501478],
       ...,
       [0.02423677, 0.00519991, 0.00790486, ..., 1.        , 0.01709267,
        0.01395516],
       [0.02384061, 0.0074029 , 0.04362236, ..., 0.01709267, 1.        ,
        0.0197809 ],
       [0.03669008, 0.01574003, 0.08501478, ..., 0.01395516, 0.0197809 ,
        1.        ]])

In [228]:
similarity.shape

(11534, 11534)

In [229]:
# getting the movie name from the user

ProductName = input(' Enter your favourite product name : ')

 Enter your favourite product name : shorts


In [230]:
# creating a list with all the movie name given in the dataset

list_of_all_titles = df['ProductName'].tolist()
list_of_all_titles

['dkny unisex black & grey printed medium trolley bag',
 'ethnovogue  beige & grey made to measure custom made kurta set with jacket',
 'spykar  pink alexa super skinny fit high-rise clean look stretchable cropped jeans',
 'raymond  blue self-design single-breasted bandhgala suit',
 'parx  brown & off-white slim fit printed casual shirt',
 'showoff  brown solid slim fit regular shorts',
 'parx  blue slim fit checked casual shirt',
 'spykar  burgundy alexa super skinny fit high-rise clean look stretchable ankle jeans',
 'parx  brown tapered fit solid regular trousers',
 'dkny unisex black large trolley bag',
 'kenneth cole  navy blue solid backpack',
 'parx  green printed polo collar t-shirt',
 'vishudh  maroon printed kurta with palazzos',
 'parx  red slim fit checked casual shirt',
 'being human  navy printed casual slim shirt',
 'homesake  & white solid handcrafted table lamp with shade',
 'sej by nisha gupta  of 6 mustard & blue printed table placemats',
 'yak yak  camel brown solid

In [231]:
# finding the close match for the movie name gievn by the user

find_close_match = difflib.get_close_matches(ProductName, list_of_all_titles)
find_close_match

['kappa  shorts']

In [232]:
close_match = find_close_match[0]
close_match

'kappa  shorts'

In [233]:
fashion_data

,ProductID,ProductName,ProductBrand,Gender,Price (INR),NumImages,Description,PrimaryColor,index
0,10017413,dkny unisex black & grey printed medium trolle...,dkny,unisex,11745,7,"black and grey printed medium trolley bag, sec...",black,0
1,10016283,ethnovogue beige & grey made to measure custo...,ethnovogue,women,5810,7,beige & grey made to measure kurta with churid...,beige,1
2,10009781,spykar pink alexa super skinny fit high-rise ...,spykar,women,899,7,pink coloured wash 5-pocket high-rise cropped ...,pink,2
3,10015921,raymond blue self-design single-breasted band...,raymond,men,5599,5,blue self-design bandhgala suitblue self-desig...,blue,3
4,10017833,parx brown & off-white slim fit printed casua...,parx,men,759,5,"brown and off-white printed casual shirt, has ...",white,4
...,...,...,...,...,...,...,...,...,...
12485,10261595,laceandme black lace non-wired lightly padded ...,laceandme,women,569,5,black lace full-coverage bralette bra lightly ...,red,12485
12486,10262843,pepe jeans men black hammock slim fit low-rise...,pepe jeans,men,1299,7,"black dark wash 5-pocket low-rise jeans, clean...",black,12486
12487,10261721,mochi women gold-toned solid heels,mochi,women,1990,5,"a pair of gold-toned open toe heels, has regul...",gold,12487
12488,10261607,612 league girls navy blue & white printed reg...,612 league,girls,602,4,navy blue and white printed mid-rise denim sho...,blue,12488


In [235]:
# finding the index of the movie with title

index_of_the_product = df[df.ProductName== close_match]['index'].values[0]
index_of_the_product

4282

In [236]:
# getting the list of similar movies

similarity_score = list(enumerate(similarity[index_of_the_product]))
similarity_score

[(0, 0.032420067610539303),
 (1, 0.0156256254558651),
 (2, 0.008591792675416194),
 (3, 0.009223220153886044),
 (4, 0.2775230867583848),
 (5, 0.004131872894003362),
 (6, 0.023501878364124623),
 (7, 0.008950026105923528),
 (8, 0.0),
 (9, 0.006919077303102061),
 (10, 0.0),
 (11, 0.19650873894364365),
 (12, 0.11949623032481832),
 (13, 0.02170252539404338),
 (14, 0.06740744638015589),
 (15, 0.0262855742450748),
 (16, 0.03998471831457821),
 (17, 0.05249469272227087),
 (18, 0.02434066802725237),
 (19, 0.023806625430079446),
 (20, 0.02071496311836871),
 (21, 0.010735579154711919),
 (22, 0.0),
 (23, 0.0),
 (24, 0.0),
 (25, 0.09490344831201283),
 (26, 0.031736833824464086),
 (27, 0.13239262575009703),
 (28, 0.004137445592046137),
 (29, 0.10654990045883544),
 (30, 0.0),
 (31, 0.005231849600426247),
 (32, 0.1245192127328502),
 (33, 0.023959098857306918),
 (34, 0.005315457159387843),
 (35, 0.1867416694484022),
 (36, 0.02325844514376037),
 (37, 0.008507152403607244),
 (38, 0.03798452774669841),
 (39

In [237]:
# sorting the clothes based on their similarity score

sorted_similar_cloth = sorted(similarity_score, key = lambda x:x[1], reverse = True)

sorted_similar_cloth

[(4197, 1.0),
 (4282, 1.0),
 (4219, 0.9754021155587186),
 (4134, 0.8421590419164211),
 (4298, 0.8421590419164211),
 (3014, 0.8278339461524109),
 (2909, 0.6108038236756567),
 (2974, 0.5731667556481271),
 (10934, 0.5473905204012451),
 (3857, 0.5380443606952293),
 (2802, 0.5337395848648472),
 (3463, 0.5337395848648472),
 (9797, 0.5266784226609396),
 (10003, 0.5266784226609396),
 (10434, 0.5259254619675926),
 (2979, 0.5226751122201141),
 (2980, 0.5226751122201141),
 (2983, 0.520460535877464),
 (3018, 0.520460535877464),
 (3240, 0.520460535877464),
 (3236, 0.516557273091438),
 (3062, 0.5161406551878406),
 (2908, 0.5123168544971214),
 (3273, 0.5123168544971214),
 (3210, 0.510273310886303),
 (2790, 0.5069543905145528),
 (9338, 0.5027407376063009),
 (3545, 0.49729015268201554),
 (5013, 0.4847833463339556),
 (2904, 0.4809992627380355),
 (10188, 0.48048289964987456),
 (10563, 0.47049700257638943),
 (3038, 0.4670075423060625),
 (379, 0.46696248499473164),
 (9762, 0.46583744904554475),
 (466, 0.46

In [239]:
print('Clothes suggested for you: \n')

i =1
for cloth in sorted_similar_cloth:
    index = cloth[0]
    title_from_index = df[df.index==index]['ProductName'].values
    if(i<30):
        print(i, '.', title_from_index)
        i+=1

Clothes suggested for you: 

1 . ['archies  gifts white 2-pieces printed ceramic cups set']
2 . ['kappa  shorts']
3 . ['lavie  black & gold-toned colourblocked t-strap flats']
4 . ['calvin klein unisex black textured medium trolley bag']
5 . ['mast & harbour  olive green & off white printed longline open front jacket']
6 . ['elle  pink pure cotton printed top']
7 . ['code by lifestyle  pink & white slim fit striped formal shirt']
8 . ['casio  women beige multi-dial watch she-3058pgl-9audf - sx255']
9 . ['pepe jeans  navy blue polo collar t-shirt']
10 . ['belle fille  tailored jacket']
11 . ['kraus jeans  blue skinny fit mid-rise ankle length stretchable cropped jeans k4050']
12 . ['wintage  black solid woven nehru jacket']
13 . ['united colors of benetton  orange & black colourblocked thong flip-flops']
14 . ['flying machine  orange & navy blue printed sweatshirt']
15 . ['van heusen  grey & brown checked slim-fit single-breasted formal suit']
16 . ['newton  silver-toned analogue watch 

In [248]:
import pickle

#define a patch for the pickle file on your disk
pick_path = 'titles.pkl'

#convert the dictionary to pickle
with open (pick_path, 'wb') as pick:
    pickle.dump(list_of_all_titles, pick)
    print("DUMPED!!")

DUMPED!!


In [249]:
pick_path = 'similarity.pkl'

#convert the dictionary to pickle
with open (pick_path, 'wb') as pick:
    pickle.dump(similarity_score, pick)
    print("DUMPED!!")

DUMPED!!


In [250]:
pick_path = 'df.pkl'

#convert the dictionary to pickle
with open (pick_path, 'wb') as pick:
    pickle.dump(df, pick)
    print("DUMPED!!")

DUMPED!!


# Cloth Recommendation

In [255]:
product_name = input(' Enter your cloth name : ')

list_of_all_titles = df['ProductName'].tolist()

find_close_match = difflib.get_close_matches(product_name, list_of_all_titles)

close_match = find_close_match[0]

index_of_the_product = df[df.ProductName== close_match]['index'].values[0]

similarity_score = list(enumerate(similarity[index_of_the_product]))

sorted_similar_products = sorted(similarity_score, key = lambda x:x[1], reverse = True)

print('Clothes suggested for you: \n')

i =1
for clothes in sorted_similar_products:
    index = clothes[0]
    title_from_index = df[df.index==index]['ProductName'].values
    if(title_from_index==0):
        continue
    title_from_index = title_from_index[0]
    
    if(i<30):
        print(i, '.', title_from_index)
        i+=1

 Enter your cloth name : pepe jeans men black hammock slim fit low-rise clean


IndexError: index 12486 is out of bounds for axis 0 with size 11534

In [ ]:
import pickle

#define a patch for the pickle file on your disk
pick_path = 'C:\\temp\\perf_data.pkl'

#convert the dictionary to pickle
with open (pick_path, 'wb') as pick:
    pickle.dump(performance_dict, pick)